## BETTER WAY 15 - 클로저 변수 스코프와 작용하는 방법을 알자
## Item15 - Know How Closures Interact with Variable Scope

숫자 리스트를 정렬할 때 특정 그룹의 숫자들이 먼저 오도록 우선순위를 매기려고 할때, 아래의 예제처럼 리스트의 `sort()`메소드에 헬퍼함수를 `key`인수로 넘기는 방법이 있다. 헬퍼의 반환값(여기서는 튜플)은 리스트에 있는 각 아이템을 정렬하는 값으로 사용된다.

In [1]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            print((0, x))
            return (0, x)
        print((1, x))
        return (1, x)
    values.sort(key=helper)

In [2]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

(1, 8)
(0, 3)
(1, 1)
(0, 2)
(0, 5)
(1, 4)
(0, 7)
(1, 6)
[2, 3, 5, 7, 1, 4, 6, 8]


위의 코드를 보고 함수안에 함수를 쓰는 것을 처음 보았다... 이러한 개념을 이해하려면 먼저, 파이썬의 **퍼스트클래스 함수(First-class function)**에 대해 알아야 한다. [School of web](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8D%BC%EC%8A%A4%ED%8A%B8%ED%81%B4%EB%9E%98%EC%8A%A4-%ED%95%A8%EC%88%98-first-class-function/)을 참고하였다. <br />

## 퍼스트클래스 함수 (First-class function)
**퍼스트클래스 함수**란 프로그래밍 언어가 함수(function)를 *first-class object* 로 취급하는 것을 의미한다. 즉, 함수 자체를 **인자(argument)**로써 다른 함수에 전달하거나 다른 함수의 결과값으로 **리턴할 수** 있고, 함수를 변수에 **할당**하거나 데이터 구조안에 **저장**할 수 있는 함수를 말한다. 아래의 예제 코드들을 통해 퍼스트클래스 함수에 대해 자세히 알아보도록 하자.

In [3]:
def square(x):
    return x * x

print(square(5))

f = square

print(square)
print(f)

25
<function square at 0x000001EC1CC068C8>
<function square at 0x000001EC1CC068C8>


위의 코드를 보면 `square`함수와 `f` 변수에 `square`를 할당한 후 출력한 결과, 같은 메모리에 할당되어 있는것을 볼 수 있다. 또한, 아래의 코드를 통해 `square`함수를 할당받은 `f`변수가 함수처럼 사용되는 것을 확인할 수 있다.

In [4]:
def square(x):
    return x * x

f = square

print(f(5))

25


이번에는 함수 자체를 인자(argument)로써 다른 함수에 전달하는 코드를 살펴보겠다. my_map 함수에 `square`함수를 인자로 전달한 후 `for`루프안에서 `square`함수를 호출하는 것을 볼 수 있다. 

In [5]:
def square(x):
    return x * x

def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))  # square 함수 호출
    return result

num_list = [1, 2, 3, 4, 5]
squares = my_map(square, num_list)  # square 함수를 인자로 전달
print(squares)

[1, 4, 9, 16, 25]


위의 코드를 `simple_square`함수 하나로 아래와 같이 나타낼 수 있다. 리스트 컴프리헨션을 이용해 간단하게 나타낼 수 있다. 간단한 함수 하나만 실행할 때는 아래와 같이 사용할 수 있다. <br />

In [6]:
def simple_square(arg_list):
    result = [num * num for num in arg_list]
    return result

num_list = [1, 2, 3, 4, 5]
simple_squares = simple_square(num_list)
print(simple_squares)

[1, 4, 9, 16, 25]


그럼 굳이 왜 퍼스트클래스 함수를 사용하는 걸까? 라는 의문이 들 수 있다. 하지만 아래의 예제를 보면 퍼스트클래스 함수의 장점을 바로 확인할 수 있다. <br />
퍼스트클래스 함수의 장점은 *이미 정의된 여러 함수를 간단히 재활용* 할 수 있다는 장점이 있다.

In [7]:
def square(x):
    return x * x

def cube(x):
    return x * x * x

def quad(x):
    return x * x * x * x

def my_map(func, arg_list):
    result = [func(i) for i in arg_list]  # square 함수 호출
    return result

num_list = [1, 2, 3, 4, 5]

squares = my_map(square, num_list)
cubes = my_map(cube, num_list)
quads = my_map(quad, num_list)

print('squares : {}'.format(squares))
print('cubes : {}'.format(cubes))
print('quads : {}'.format(quads))

squares : [1, 4, 9, 16, 25]
cubes : [1, 8, 27, 64, 125]
quads : [1, 16, 81, 256, 625]


위의 예제와 같이 `square, cube, quad`의 정의되어 있는 함수를 `my_map`과 같은 wrapper 함수를 정의해 편리하게 쓸 수 있다.

이번에는 함수의 결과값으로 다른 함수를 리턴하는 방법에 대해 알아보자

In [8]:
def logger(msg):
    
    def log_message():  # 1
        print('Log : {}'.format(msg))
    
    return log_message

log_hi = logger('Hi')
print(log_hi)  # log_message 오브젝트가 출력됨
log_hi()  # Log: Hi 가 출력됨

<function logger.<locals>.log_message at 0x000001EC1CC06048>
Log : Hi


위의 예제에서 정의된 `log_message`함수를 `logger` 함수의 리턴값으로 리턴하여 `log_hi`라는 변수에 할당한 후 호출한 것을 볼 수 있다. 그런데 여기서 신기한 점을 발견할 수 있다. `msg`와 같은 함수의 지역변수(local variable)의 값은 함수가 호출된 이후에 메모리에서 사라지므로 다시 참조할 수 없다. 하지만, `msg` 변수에 할당했던 'Hi'값이  `logger`함수가 종료된 이후에도 참조가 되었다. <br />
`log_message`와 같은 함수를 **클로저(closure)**라고 하며, 클로저는 다른 함수의 지역변수를 그 함수가 종료된 이후에도 기억할 수 있다.

In [9]:
def logger(msg):
    
    def log_message():  # 1
        print('Log :', msg)
        
    return log_message

log_hi = logger('Hi')
print(log_hi)  # log_message 오브젝트가 출력됨
log_hi()  # Log : Hi 가 출력됨

del logger  # logger 오브젝트를 지움

# logger 오브젝트가 지워진 것을 확인
try:
    print(logger)
except NameError as e:
    print('NameError :',e)
    
log_hi()  # logger가 지워진 뒤에도 Log: Hi 가 출력됨

<function logger.<locals>.log_message at 0x000001EC1CC06510>
Log : Hi
NameError : name 'logger' is not defined
Log : Hi


위의 예제에서 `logger`가 지워진 뒤에도 `log_hi()`를 실행하여 `log_message`가 호출된 것을 확인할 수 있다.

In [10]:
def simple_html_tag(tag, msg):
    print('<{tag}>{msg}<{tag}>'.format(tag=tag, msg=msg))
    
simple_html_tag('h1', '심플 헤딩 타이틀')

print('-' * 30)

# 함수를 리턴하는 함수
def html_tag(tag):
    
    def wrap_text(msg):
        print('<{tag}>{msg}<{tag}>'.format(tag=tag, msg=msg))
        
    return wrap_text

print_h1 = html_tag('h1')  # `
print(print_h1)  # 2
print_h1('첫 번째 헤딩 타이틀')  # 3
print_h1('두 번째 헤딩 타이틀')  # 4

print_p = html_tag('p')
print_p('이것은 paragraph 입니다.')

<h1>심플 헤딩 타이틀<h1>
------------------------------
<function html_tag.<locals>.wrap_text at 0x000001EC1CC2F158>
<h1>첫 번째 헤딩 타이틀<h1>
<h1>두 번째 헤딩 타이틀<h1>
<p>이것은 paragraph 입니다.<p>


`#1`에서 `html_tag` 함수를 `print_h1` 변수에 할당한 후 `#2`에서 변수의 값을 출력했을 때, `wrap_text`함수 오브젝트가 할당되어 있는 것을 볼 수 있다. 그리고 `#3`과 `#4`에서 `wrap_text`함수를 호출한 것을 볼 수 있다. 

## 클로저 - Closure
> 프로그래밍 언어에서의 클로저란 퍼스트클래스 함수를 지원하는 언어의 네임 바인딩 기술이다. 클로저는 어떤 함수를 함수 자신이 가지고 있는 환경과 함께 저장한 레코드이다. 또한 함수가 가진 프리변수(free variable)를 클로저가 만들어지는 당시의 값과 레퍼런스에 맵핑하여 주는 역할을 한다. 클로저는 일반 함수와는 다르게, 자신의 영역 밖에서 호출된 함수의 변수값과 레퍼런스를 복사하고 저장한 뒤, 이 캡처한 값들에 액세스할 수 있게 도와준다.
> - 출처: [SCHOOL OF WEB](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%81%B4%EB%A1%9C%EC%A0%80-closure/)

In [11]:
def outer_func():  # 1
    message = 'Hi'  # 3
    
    def inner_func():  # 4
        print(message)  # 6
        
    return inner_func()  # 5

outer_func()  # 2

Hi


위의 코드의 프로세스를 확인해보면 <br />
1. `#1`에서 정의된 함수 `outer_func`을 `#2`에서 호출한다. 
2. `outer_func`가 실행된 후, `#3`에서 `message`라는 변수에 'Hi' 문자열을 할당한다.
3. `#4`에서 `inner_func()`을 정의하고 `#5`에서 `inner_func`을 호출과 동시에 리턴한다.
4. `#6`에서 `message`변수를 참조하여 출력한다. 여기서 `message`는 `inner_func`안에 정의 되어있지 않지만, `inner_func`안에서 사용되기 때문에 이를 *프리변수* 라고 한다.

In [12]:
def outer_func():  # 1
    message = 'Hi'  # 3
    
    def inner_func():  # 4
        print(message)  # 6
        
    return inner_func  # 5 <-- ()를 지움

outer_func()  # 2

<function __main__.outer_func.<locals>.inner_func>

위의 코드는 `#5`에서 `()`를 지운 코드이다. 여기서는 `inner_func` 함수 오브젝트를 리턴한다.

In [13]:
def outer_func():  # 1
    message = 'Hi'  # 3
    
    def inner_func():  # 4
        print(message)  # 6
        
    return inner_func  # 5 

my_func = outer_func()  # 2
print("object: ", my_func) # 7
my_func() # 8

object:  <function outer_func.<locals>.inner_func at 0x000001EC1CC18620>
Hi


위의 코드는 `#7`에서 `my_func`이라는 변수에 `inner_func` 오브젝트 변수를 할당한 코드이다. 할당한 변수를 `#8` 에서 호출 하니 'Hi'라는 문자열이 출력되었다. `outer_func`는 `#2`에서 호출된 후, 종료되었지만, `#8` 에서 `my_func` 함수가 `outer_func`함수의 로컬변수인 `message`를 참조했다. 이것이 가능한 이유가 바로 **클로저** 때문이다. 클로저는 함수의 프리변수 값을 따로 저장해 놓는다.

In [14]:
def outer_func():  # 1
    message = 'Hi'  # 3
    
    def inner_func():  # 4
        print(message)  # 6
        
    return inner_func  # 5 

my_func = outer_func()  # 2

print('Object :\n', my_func)
print()
print('Directory :\n', dir(my_func))
print()
print('Type :\n', type(my_func.__closure__))
print()
print('my_func.__closure__ :\n', my_func.__closure__)
print()
print('my_func.__closure__[0] :\n', my_func.__closure__[0])
print()
print('dir(my_func.__closure__[0]) :\n', dir(my_func.__closure__[0]))
print()
print('my_func.__closure__[0].cell_contents :\n', my_func.__closure__[0].cell_contents)

Object :
 <function outer_func.<locals>.inner_func at 0x000001EC1CBECD90>

Directory :
 ['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

Type :
 <class 'tuple'>

my_func.__closure__ :
 (<cell at 0x000001EC1CBC5D98: str object at 0x000001EC1CC1AC38>,)

my_func.__closure__[0] :
 <cell at 0x000001EC1CBC5D98: str object at 0x000001EC1CC1AC38>

dir(my_func.__closure__[0]) :
 ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__

아래의 예제는 클로저를 이용해 하나의 함수로 여러가지 함수를 만들 수 있다는 것을 보여주는 간단한 예제이다. 

In [15]:
def outer_func(tag):
    tag = tag
    
    def inner_func(txt):
        text = txt
        print('<{tag}>{text}<{tag}>'.format(tag=tag, text=text))
        
    return inner_func

h1_func = outer_func('h1')
p_func = outer_func('p')

h1_func('h1 태그의 안입니다')
p_func('p태그의 안입니다')

<h1>h1 태그의 안입니다<h1>
<p>p태그의 안입니다<p>


---

퍼스트클래스 함수와 클로저에 대해 알아보았으니, 다시 교재로 돌아오면,

In [16]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [17]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


위의 예제 코드의 동작 이유는 아래와 같다.
* 파이썬은 클로저(closure)를 지원한다. 클로저에 대한 정의는 위의 설명을 참고하자. 클로저는 `sort_priority` 의 프리변수(`values`, `group`) 접근할 수 있다.
* 함수는 파이썬에서 **일급 객체(first-class object)** 이다. 이 의미는 위의 퍼스트클래스 함수에서도 설명 했지만, 함수를 직접 참조하고, 변수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if 문 등에서 비교할 수 있다는 의미이다. 따라서 `sort`메소드에 클로저 함수를 `key`로 받을 수 있다.
* 파이썬에는 튜플을 비교하는 특정한 규칙이 있다. 먼저 인덱스 0으로 아이템을 비교하고, 그다음으로 인덱스 1, 다음은 인덱스 2 와 같이 비교한다.

아래의 코드는 우선순위가 높은 아이템을 발션했는지 여부를 `foud`라는 Boolean 변수를 추가한 코드다.

In [18]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return(0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [19]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

found = sort_priority2(numbers, group)
print('Found :' ,found)
print(numbers)

Found : False
[2, 3, 5, 7, 1, 4, 6, 8]


하지만, 출력을 보면 정렬된 결과는 올바르지만 `false`의 결과는 `True`가 아니다. 그 이뉴는 표현식에서 변수를 참조할 때 파이썬 인터프리터는 참조를 해결하기 위해 다음의 순서로 **스코프**(scope: 유효범위)를 탐색한다. <br />
1. 현재 함수의 스코프
2. (현재 스코프를 담고 있는 다른 함수 같은) 감싸고 있는 스코프
3. 코드를 포함하고 있는 모듈의 스코프(전역 스코프라고 함)
4. (`len`이나 `str`같은 함수를 담고 있는) 내장 스코프

따라서, 변수 `found`는 `sort_priority2`함수가 아닌 `helper`클로저에서 `True`로 할당되기 때문에 `helper` 클로저 안에서 일어나는 새 변수 정의로 처리된다.

### 데이터 얻어오기
파이썬3 에서는 클로저에서 데이터를 얻어오는 특별한 문법이 있다. `nonlocal` 문은 특정 변수 이름에 할당할 때 스코프 탐색이 일어나야 함을 나타낸다. `nonlocal`은 모듈 수준 스코프까지는 탐색할 수 없게 한다.

In [20]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [21]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

found = sort_priority3(numbers, group)
print('Found :' ,found)
print(numbers)

Found : True
[2, 3, 5, 7, 1, 4, 6, 8]


### 정리
* 클로저 함수는 자신이 정의된 스코프 중 어디에 있는 변수도 참조할 수 있다.
* 기본적으로 클로저에 변수를 할당하면 바깥쪽 스코프에는 영향을 미치지 않는다.
* 파이썬3에는 `nonlocal` 문을 사용하여 클로저를 감싸고 있는 스코프의 변수를 수정할 수 있다.
* 간단한 함수 외에는 `nonlocal`을 사용하지 않는 것이 좋다.